In [18]:
from pysheds.grid import Grid
import numpy as np
import rasterio

path = "../../assets/dem/clipped.tif"

In [19]:
output_tif = '../../assets/dem/output_float.tif'

with rasterio.open(path) as src:
    data = src.read(1).astype('float32')  # Read and convert to float
    profile = src.profile
    profile.update(dtype='float32')  # Update the profile data type

    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(data, 1)

In [20]:
grid = Grid.from_raster(output_tif)

In [21]:
grid

'affine' : Affine(0.0008333333333333296, 0.0, -84.9425,
       0.0, -0.0008333333333333322, 34.29333333333334)
'shape' : (1831, 2412)
'nodata' : np.float32(-32768.0)
'crs' : <Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

'mask' : array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [22]:
grid.nodata

np.float32(-32768.0)

In [23]:
dem = grid.read_raster(path)

In [24]:
dem

Raster([[237, 231, 228, ..., 219, 220, 219],
        [230, 228, 229, ..., 215, 216, 214],
        [227, 225, 229, ..., 210, 213, 211],
        ...,
        [257, 254, 256, ..., 101, 102, 104],
        [259, 257, 259, ...,  96,  98, 102],
        [265, 263, 262, ..., 100,  99, 103]], dtype=int16)

In [25]:
dem.dtype

dtype('int16')

In [26]:
pit_filled_dem = grid.fill_pits(dem, nodata_in=-32768, nodata_out=-32768)
pit_filled_dem

Raster([[237., 231., 228., ..., 219., 220., 219.],
        [230., 228., 229., ..., 215., 216., 214.],
        [227., 227., 229., ..., 210., 213., 211.],
        ...,
        [257., 254., 256., ..., 101., 102., 104.],
        [259., 257., 259., ...,  96.,  98., 102.],
        [265., 263., 262., ..., 100.,  99., 103.]])

In [27]:
pit_filled_dem.dtype

dtype('float64')

In [28]:
flooded_dem = grid.fill_depressions(pit_filled_dem)
flooded_dem

Raster([[237., 231., 228., ..., 219., 220., 219.],
        [230., 228., 229., ..., 215., 216., 214.],
        [227., 227., 229., ..., 211., 213., 211.],
        ...,
        [257., 254., 256., ..., 101., 102., 104.],
        [259., 257., 259., ...,  96.,  98., 102.],
        [265., 263., 262., ..., 100.,  99., 103.]])

In [29]:
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

In [30]:
inflated_dem

Raster([[237.     , 231.     , 228.     , ..., 219.     , 220.     ,
         219.     ],
        [230.     , 228.     , 229.     , ..., 215.     , 216.     ,
         214.     ],
        [227.00002, 227.00004, 229.     , ..., 211.00006, 213.     ,
         211.00002],
        ...,
        [257.     , 254.     , 256.     , ..., 101.     , 102.     ,
         104.     ],
        [259.     , 257.     , 259.     , ...,  96.     ,  98.     ,
         102.     ],
        [265.     , 263.     , 262.     , ..., 100.     ,  99.     ,
         103.     ]])

In [31]:
# Compute flow directions
fdir = grid.flowdir(inflated_dem)

TypeError: `nodata` value not representable in dtype of array.